In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.5 MB/s eta 0:00:00


## GPT 입력값 만들기

GPT 입력값을 만들려면 토크나이저부터 준비해야 하며 코드3을 수행하면 GPT 모델이 사용하는 토크나이저를 초기화할 수 있다. 먼저 자신의 구글 드라이브 경로(/gdrive/My Drive/nlpbook/bbpe)에는 이전 실습에서 만든 바이트 기준 BPE 어휘 집합(vocab.json)과 바이그램 쌍의 병합 우선순위(merge.txt)가 있어야 합니다.

In [3]:
# GPT 토크나이저 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [4]:
# 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [5]:
# GPT 모델은 바이트 기준 BPE를 적용
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [ ]:
#  GPT 모델 입력
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [ ]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
# 'input_ids'는 토큰화 결과를 가지고 각 토큰들을 인덱스(index)로 바꾼 것
batch_inputs['input_ids']

[[334, 2359, 263, 582, 4046, 463, 4025, 0, 0, 0, 0, 0],
 [3597, 336, 2880, 755, 2860, 356, 803, 421, 444, 690, 875, 2901],
 [5281, 451, 3580, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# attention_mask는 일반 토큰이 자리한 곳(1)과 패딩 토큰이 자리한 곳(0)을 구분해 알려주는 장치
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

## BERT 입력값 만들기

BERT 모델 입력값을 만들려면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 워드피스 어휘집합 구축 결과(`vocab.txt`)가 있어야 합니다. 다음을 수행해 위에서 이미 만들어 놓은 워드피스 어휘집합을 포함한 BERT 토크나이저를 `tokenizer_bert`라는 변수로 선언합니다.

In [ ]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case=False,
)

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [ ]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [ ]:
# # BERT 모델 세 가지의 입력값
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# 토큰 인덱스 시퀀스
batch_inputs['input_ids']

[[2, 622, 2639, 16, 16, 1993, 3682, 1990, 3467, 3, 0, 0],
 [2, 997, 16, 16, 16, 2596, 2045, 2809, 1981, 1078, 16, 3],
 [2, 3341, 9156, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [ ]:
# BERT의 attention_mask는 GPT와 마찬가지로 일반 토큰이 자리한 곳(1)과 패딩 토큰이 자리한 곳(0)을 구분
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [ ]:
# token_type_ids는 세그먼트(segment)에 해당하는 것으로 모두 0
# BERT 모델은 기본적으로 문서(혹은 문장) 2개를 입력받는데, token_type_ids로 구분.
# 첫 번째 세그먼트(문서 혹은 문장)에 해당하는 token_type_ids는 0, 두 번째 세그먼트는 1
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]